# __Diffusion MRI data pre-processing notebook__
#### __Last updated on:__ 27/02/2020
#### __Author:__ Rakshit Dadarwal

### __Requirements:__
#### 1. DIPY (https://dipy.org/)
#### 2. nipype (https://nipype.readthedocs.io/en/latest/)
#### 3. FSL (https://fsl.fmrib.ox.ac.uk/fsl/fslwiki)

### __This script includes:__
#### 1. Denoising
#### 2. TOPUP: Susceptibility-induced distortion correction  
#### 3. EDDY: eddy current-induced distortion and motion correction

### __Import python libraries__

In [ ]:
import os
import numpy as np
import nibabel as nib
import timeit; timeit.timeit()
# DIPY Local PCA Denoising
from dipy.denoise.localpca import mppca 
# Nipype fsl interfce
import nipype.interfaces.fsl as fsl

### __Define DWI data path__

In [ ]:
#--------------------------------------------
#        DWI data files 
#--------------------------------------------
data_path = "/home/User/Diffusion"
dwi_file = 'dMRI.nii.gz' # 4D diffusion data file name
dwi_rev_file = 'dMRI_rev.nii.gz' # dwi acquisition with reverse phase encoding
bvals_file = 'dMRI.bvals' # bval file name
bvecs_file = 'dMRI.bvecs' # bvec file name
denoised_file = 'dwi_denoised.nii.gz' # denoised output file name 

### __Denoising__

In [ ]:
# Change directory
cwdir = os.getcwd()
os.chdir(data_path)
        
#------------------------------------------------
#         DIPY MP-PCA denoising
#------------------------------------------------
img = nib.load(os.path.join(data_path,dwi_file))
data = img.get_data()    
            
denoised = mppca(data, patch_radius=2)
        
nib.save(nib.Nifti1Image(denoised, img.affine), os.path.join(data_path,denoised_file))

print('Elapsed time:',timeit.timeit())

### __TOPUP__
* Susceptibility-induced distortion correction

In [ ]:
#------------------------------------------------
#             Extract b0 image
#------------------------------------------------
def extract_b0(inImage, outImage):
    fslroi = fsl.ExtractROI(in_file=inImage,
                            roi_file=outImage, t_min=0, t_size=1)
    fslroi.run()
# extract AP b0
extract_b0(dwi_file, 'epi_b0.nii.gz')
# extract PA b0
extract_b0(dwi_rev_file, 'epi_rev_b0.nii.gz')

#------------------------------------------------
#             FSL TOPUP 
#------------------------------------------------
# data prep for topup
merger = fsl.Merge(in_files=['epi_b0.nii.gz','epi_rev_b0.nii.gz'],
                   dimension = 't', output_type='NIFTI_GZ')
merger.run()
        
file = open('topup_encoding.txt','w')
file.write('0 1 0 0.05\n0 -1 0 0.05')
file.close()
        
topup = fsl.TOPUP(in_file = 'epi_b0_merged.nii.gz',
                  encoding_file = 'topup_encoding.txt',
                  output_type = "NIFTI_GZ")
topup.run()
        
#------------------------------------------------
#            FSL ApplyTOPUP 
#------------------------------------------------
applytopup = fsl.ApplyTOPUP(in_files = ['epi_b0.nii.gz', 'epi_rev_b0.nii.gz'],
                            encoding_file = 'topup_encoding.txt',
                            in_topup_fieldcoef = "epi_b0_merged_base_fieldcoef.nii.gz",
                            in_topup_movpar = 'epi_b0_merged_base_movpar.txt',
                            output_type = "NIFTI_GZ") 
applytopup.run()
        
print('Elapsed time:',timeit.timeit())

### __EDDY__
* eddy current-induced distortion and motion correction

In [ ]:
#------------------------------------------------
#            FSL Eddy
#------------------------------------------------
# data prep for eddy
btr = fsl.BET(in_file='epi_b0_corrected.nii.gz',
              frac=0.5, out_file='brain.nii.gz', mask=True)
btr.run()

# total nuber of volumes in dwi data
img = nib.load(denoised_file).get_data()
nvolumes = img.shape[-1]

file = open('index.txt','w')
for i in range(0, nvolumes):
    file.write('1 ')
file.close()
            
eddy = fsl.Eddy(in_file = denoised_file,
                in_mask  = 'brain_mask.nii.gz',
                in_index = 'index.txt',
                in_acqp  = 'topup_encoding.txt',
                in_topup_fieldcoef = "epi_b0_merged_base_fieldcoef.nii.gz",
                in_topup_movpar = 'epi_b0_merged_base_movpar.txt',
                in_bvec  = bvecs_file,
                in_bval  = bvals_file, use_cuda = False, 
                is_shelled=True)
eddy.run()
        
print('Elapsed time:',timeit.timeit())
print('Enjoy!!')